In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from math import sqrt

In [5]:
data=pd.read_csv('C:/Users/USER/Desktop/2020 summer dacon/2020_jeju_credit card/data/20200722 이후/april_add_merge_with_homeproportion.csv',encoding = 'CP949')

In [6]:
data.shape

(1114451, 19)

In [7]:
data.head(10)

,Unnamed: 0,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,festival_count,year,month,rain_day,total_tourist,corona_confirmed,total_sale_by_indst,sex_age_population,total_home
0,1,강원,건강보조식품 소매업,강원,20s,1,1,3,27500.0,3,8,2019,8,13.9,1603895,0,33849.0,100115,0.57
1,2,강원,건강보조식품 소매업,강원,20s,1,1,3,139000.0,3,8,2019,6,10.6,1374767,0,33849.0,100115,0.72
2,3,강원,건강보조식품 소매업,강원,20s,1,1,3,395500.0,3,11,2019,9,9.8,2062151,0,33849.0,100115,0.72
3,4,강원,건강보조식품 소매업,강원,20s,1,1,3,427510.0,2,0,2020,3,7.3,761521,29,33849.0,100115,0.82
4,5,강원,건강보조식품 소매업,강원,20s,1,1,3,605000.0,3,4,2019,2,4.7,2146566,0,33849.0,100115,0.70
5,6,강원,건강보조식품 소매업,강원,20s,1,1,4,311200.0,4,4,2019,1,4.1,2086269,0,33849.0,100115,0.69
6,7,강원,건강보조식품 소매업,강원,20s,1,1,6,168500.0,6,0,2020,1,4.1,2086269,0,33849.0,100115,0.68
7,8,강원,건강보조식품 소매업,강원,20s,1,1,6,478500.0,5,1,2020,4,9.7,861476,17,33849.0,100115,0.82
8,9,강원,건강보조식품 소매업,강원,20s,1,1,13,843100.0,16,0,2019,11,9.1,1808904,0,33849.0,100115,0.75
9,10,강원,건강보조식품 소매업,강원,20s,2,1,3,72000.0,3,7,2019,10,8.5,2291278,0,33849.0,78526,0.70


In [11]:
# DataFrame.dtypes for data must be int, float or bool.
# Did not expect the data types in the following fields
# : CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE
cat_features = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM','AGE']
for i in enumerate (cat_features) : 
    ca = i[1] 
    data[ca] = data[ca].astype('category') 

In [14]:
X = data.drop(columns=['AMT'])
y = data['AMT']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False,random_state=2020)
X_train.shape

(780115, 18)

In [15]:
X_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]
X_test.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_test.columns]

lightgbm

In [ ]:
params_grid = {
    'num_leaves': [31, 127],
    'reg_alpha': [0.1, 0.5],
    'min_data_in_leaf': [30, 50, 100, 300, 400],
    'lambda_l1': [0, 1, 1.5],
    'lambda_l2': [0, 1]
    }
lgb = LGBMRegressor(boosting_type='gbdt', num_boost_round=2000, learning_rate=0.01)
lgb_grid = GridSearchCV(estimator=lgb,
                        param_grid=params_grid,
                        n_jobs=10,
                        verbose=3)
lgb_grid.fit(X_train,y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
C:\Users\USER\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [ ]:
lgb_grid.best_params_

In [ ]:
y_pred4 = lgb_grid.predict(X_test)
lgb_rmse = sqrt(mean_squared_error(y_test, y_pred4))
lgb_rmse

In [ ]:
lgb_feature_importance = lgb_grid.best_estimator_.feature_importances_
lgb_feature_imp = pd.Series(lgb_feature_importance,index=X_train.columns).sort_values(ascending=False)
lgb_feature_imp

In [ ]:
plt.figure(figsize=(12,10))
plt.title("LGB feature Importance")
sns.barplot(x=lgb_feature_imp[0:30], y=lgb_feature_imp.index[0:30])